This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-tag-management).

## Wallaroo SDK Tag Tutorial

The following tutorial demonstrates how to use Wallaroo Tags.  Tags are applied to either model versions or pipelines.  This allows organizations to track different versions of models, and search for what pipelines have been used for specific purposes such as testing versus production use.

The following will be demonstrated:

* List all tags in a Wallaroo instance.
* List all tags applied to a model.
* List all tags applied to a pipeline.
* Apply a tag to a model.
* Remove a tag from a model.
* Apply a tag to a pipeline.
* Remove a tag from a pipeline.
* Search for a model version by a tag.
* Search for a pipeline by a tag.

This demonstration provides the following through the Wallaroo Tutorials Github Repository:

* `models/ccfraud.onnx`: a sample model used as part of the [Wallaroo 101 Tutorials](https://docs.wallaroo.ai/wallaroo-101/).

## Steps

The following steps are performed use to connect to a Wallaroo instance and demonstrate how to use tags with models and pipelines.

### Load Libraries

The first step is to load the libraries used to connect and use a Wallaroo instance.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError

### Connect to Wallaroo

The following command is used to connect to a Wallaroo instance from within a Wallaroo Jupyter Hub service.  For more information on connecting to a Wallaroo instance, see the [Wallaroo SDK Guides](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/).

In [2]:
wl = wallaroo.Client()

### Set Variables

The following variables are used to create or connect to existing workspace and pipeline.  The model name and model file are set as well.  Adjust as required for your organization's needs.

The methods `get_workspace` and `get_pipeline` are used to either create a new workspace and pipeline based on the variables below, or connect to an existing workspace and pipeline with the same name.  Once complete, the workspace will be set as the current workspace where pipelines and models are used.

In [3]:
workspace_name = 'tagtestworkspace'
pipeline_name = 'tagtestpipeline'
model_name = 'tagtestmodel'
model_file_name = './models/ccfraud.onnx'

In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [5]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'tagtestworkspace', 'id': 2, 'archived': False, 'created_by': '0bbf2f62-a4f1-4fe5-aad8-ec1cb7485939', 'created_at': '2022-11-29T17:14:57.185365+00:00', 'models': [], 'pipelines': []}

### Upload Model and Create Pipeline

The `tagtest_model` and `tagtest_pipeline` will be created (or connected if already existing) based on the variables set earlier.

In [6]:
tagtest_model = wl.upload_model(model_name, model_file_name).configure()
tagtest_model

{'name': 'tagtestmodel', 'version': '70169e97-fb7e-4922-82ba-4f5d37e75253', 'file_name': 'ccfraud.onnx', 'image_path': None, 'last_update_time': datetime.datetime(2022, 11, 29, 17, 15, 21, 703465, tzinfo=tzutc())}

In [7]:
tagtest_pipeline = get_pipeline(pipeline_name)
tagtest_pipeline

name,tagtestpipeline
created,2022-11-29 17:15:21.785352+00:00
last_updated,2022-11-29 17:15:21.785352+00:00
deployed,(none)
tags,
versions,5a4ff3c7-1a2d-4b0a-ad9f-78941e6f5677
steps,


### List Pipeline and Model Tags

This tutorial assumes that no tags are currently existing, but that can be verified through the Wallaroo client `list_pipelines` and `list_models` commands.  For this demonstration, it is recommended to use unique tags to verify each example.

In [8]:
wl.list_pipelines()

name,created,last_updated,deployed,tags,versions,steps
tagtestpipeline,2022-29-Nov 17:15:21,2022-29-Nov 17:15:21,(unknown),,5a4ff3c7-1a2d-4b0a-ad9f-78941e6f5677,


In [9]:
wl.list_models()

Name,# of Versions,Owner ID,Last Updated,Created At
tagtestmodel,1,"""""",2022-11-29 17:15:21.703465+00:00,2022-11-29 17:15:21.703465+00:00


### Create Tag

Tags are created with the Wallaroo client command `create_tag(String tagname)`.  This creates the tag and makes it available for use.

The tag will be saved to the variable `currentTag` to be used in the rest of these examples.

In [10]:
# Now we create our tag
currentTag = wl.create_tag("My Great Tag")

### List Tags

Tags are listed with the Wallaroo client command `list_tags()`, which shows all tags and what models and pipelines they have been assigned to.

In [11]:
# List all tags

wl.list_tags()

[]

### Assign Tag to a Model

Tags are assigned to a model through the Wallaroo Tag `add_to_model(model_id)` command, where `model_id` is the model's numerical ID number.  The tag is applied to the most current version of the model.

For this example, the `currentTag` will be applied to the `tagtest_model`.  All tags will then be listed to show it has been assigned to this model.

In [12]:
# add tag to model

currentTag.add_to_model(tagtest_model.id())

{'model_id': 1, 'tag_id': 1}

In [13]:
# list all tags to verify

wl.list_tags()

id,tag,models,pipelines
1,My Great Tag,"[('tagtestmodel', ['70169e97-fb7e-4922-82ba-4f5d37e75253'])]",[]


### Search Models by Tag

Model versions can be searched via tags using the Wallaroo Client method `search_models(search_term)`, where `search_term` is a string value.  All models versions containing the tag will be displayed.  In this example, we will be using the text from our tag to list all models that have the text from `currentTag` in them.

In [14]:
# Search models by tag

wl.search_models('My Great Tag')

name,version,file_name,image_path,last_update_time
tagtestmodel,70169e97-fb7e-4922-82ba-4f5d37e75253,ccfraud.onnx,None,2022-11-29 17:15:21.703465+00:00


### Remove Tag from Model

Tags are removed from models using the Wallaroo Tag `remove_from_model(model_id)` command.

In this example, the `currentTag` will be removed from `tagtest_model`.  A list of all tags will be shown with the `list_tags` command, followed by searching the models for the tag to verify it has been removed.

In [15]:
### remove tag from model

currentTag.remove_from_model(tagtest_model.id())

{'model_id': 1, 'tag_id': 1}

In [16]:
# list all tags to verify it has been removed from `tagtest_model`.

wl.list_tags()

[]

In [17]:
# search models for currentTag to verify it has been removed from `tagtest_model`.

wl.search_models('My Great Tag')

[]

### Add Tag to Pipeline

Tags are added to a pipeline through the Wallaroo Tag `add_to_pipeline(pipeline_id)` method, where `pipeline_id` is the pipeline's integer id.

For this example, we will add `currentTag` to `testtest_pipeline`, then verify it has been added through the `list_tags` command and `list_pipelines` command.

In [18]:
# add this tag to the pipeline
currentTag.add_to_pipeline(tagtest_pipeline.id())

{'pipeline_pk_id': 1, 'tag_pk_id': 1}

In [19]:
# list tags to verify it was added to tagtest_pipeline

wl.list_tags()


id,tag,models,pipelines
1,My Great Tag,[],"[('tagtestpipeline', ['5a4ff3c7-1a2d-4b0a-ad9f-78941e6f5677'])]"


In [20]:
# get all of the pipelines to show the tag was added to tagtest-pipeline

wl.list_pipelines()

name,created,last_updated,deployed,tags,versions,steps
tagtestpipeline,2022-29-Nov 17:15:21,2022-29-Nov 17:15:21,(unknown),My Great Tag,5a4ff3c7-1a2d-4b0a-ad9f-78941e6f5677,


### Search Pipelines by Tag

Pipelines can be searched through the Wallaroo Client `search_pipelines(search_term)` method, where `search_term` is a string value for tags assigned to the pipelines.

In this example, the text "My Great Tag" that corresponds to `currentTag` will be searched for and displayed.

In [21]:
wl.search_pipelines('My Great Tag')

name,version,creation_time,last_updated_time,deployed,tags,steps
tagtestpipeline,5a4ff3c7-1a2d-4b0a-ad9f-78941e6f5677,2022-29-Nov 17:15:21,2022-29-Nov 17:15:21,(unknown),My Great Tag,


### Remove Tag from Pipeline

Tags are removed from a pipeline with the Wallaroo Tag `remove_from_pipeline(pipeline_id)` command, where `pipeline_id` is the integer value of the pipeline's id.

For this example, `currentTag` will be removed from `tagtest_pipeline`.  This will be verified through the `list_tags` and `search_pipelines` command.

In [22]:
## remove from pipeline
currentTag.remove_from_pipeline(tagtest_pipeline.id())

{'pipeline_pk_id': 1, 'tag_pk_id': 1}

In [23]:
wl.list_tags()

[]

In [24]:
## Verify it was removed
wl.search_pipelines('My Great Tag')

[]